In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
model_parameters = QgParams(dynamic_T=False)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4, mode="symbolic")

In [ ]:
model_parameters.symbolic_params

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, OceanicSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicTensorLinear, SymbolicTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _symbolic_tensordot, _shift_dict_keys, _add_to_dict

In [ ]:
%%time
# Takes ~5 mins
# ocn_ip_stored = OceanicSymbolicInnerProducts(model_parameters, stored=True, return_symbolic=True)
ocn_ip = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
%%time
# Takes ~5 mins
# atm_ip_stored = AtmosphericSymbolicInnerProducts(model_parameters, return_symbolic=True)
atm_ip = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
# atm_ip_stored.save_to_file('temp_atm_sym_lin.ip')
# ocn_ip_stored.save_to_file('temp_ocn_sym_lin.ip')

In [ ]:
atm_loaded = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)
ocn_loaded = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
atm_loaded.load_from_file('temp_atm_sym_lin.ip')
ocn_loaded.load_from_file('temp_ocn_sym_lin.ip')

## Testing the symbolic_qgtensor

In [ ]:
from qgs.functions.symbolic_output import create_symbolic_equations, translate_equations, print_equations, equation_as_function

In [ ]:
%%time
funcs, = create_symbolic_equations(model_parameters, atm_loaded, ocn_loaded)

In [ ]:
ten = SymbolicTensorLinear(model_parameters, atm_loaded, ocn_loaded)

In [ ]:
ten.print_tensor(ocn_loaded._C)

In [ ]:
%%time
fx = equation_as_function(funcs, model_parameters, subs=True, language='python', string_output=True)

In [ ]:
for i in fx:
    print(i)

In [ ]:
from numba import njit

In [ ]:
@njit
def f(t, U):
	#Tendency function of the qgs model
	U_out = np.empty_like(U)
	U_out[0] = -0.05*U[0] + 0.05*U[10] + 0.168986428105931*U[21] + 0.251005505082615*U[23]
	U_out[1] = -0.980418808722262*U[0]*U[2] - 0.980418808722262*U[10]*U[12] + 0.05*U[11] - 1.56867009395562*U[13]*U[15] - 1.50055762081784*U[14]*U[17] + 1.50055762081784*U[15]*U[16] - 0.05*U[1] - 0.0112217054051163*U[20] + 0.120745727252785*U[2] - 1.56867009395562*U[3]*U[5] - 1.50055762081784*U[4]*U[7] + 1.50055762081784*U[5]*U[6]
	U_out[2] = 0.980418808722262*U[0]*U[1] + 0.980418808722262*U[10]*U[11] + 0.05*U[12] + 1.56867009395562*U[13]*U[14] + 1.50055762081784*U[14]*U[16] + 1.50055762081784*U[15]*U[17] - 0.120745727252785*U[1] + 0.05*U[24] - 0.05*U[2] + 1.56867009395562*U[3]*U[4] + 1.50055762081784*U[4]*U[6] + 1.50055762081784*U[5]*U[7]
	U_out[3] = 1.87265793760678*U[11]*U[15] - 1.87265793760678*U[12]*U[14] + 0.05*U[13] + 3.74531587521356*U[16]*U[19] - 3.74531587521356*U[17]*U[18] + 1.87265793760678*U[1]*U[5] - 0.00679432430697251*U[20] + 0.0810088839426413*U[22] - 1.87265793760678*U[2]*U[4] - 0.05*U[3] + 3.74531587521356*U[6]*U[9] - 3.74531587521356*U[7]*U[8]
	U_out[4] = -1.02902937637678*U[0]*U[5] - 1.02902937637678*U[10]*U[15] + 1.73752196836555*U[11]*U[17] + 0.574852231579351*U[12]*U[13] - 1.73752196836555*U[12]*U[16] + 0.05*U[14] + 1.73752196836555*U[1]*U[7] - 0.0164935614721478*U[21] + 0.574852231579351*U[2]*U[3] - 1.73752196836555*U[2]*U[6] - 0.05*U[4] + 0.0570836566449895*U[5]
	U_out[5] = 1.02902937637678*U[0]*U[4] + 1.02902937637678*U[10]*U[14] - 0.574852231579351*U[11]*U[13] - 1.73752196836555*U[11]*U[16] - 1.73752196836555*U[12]*U[17] + 0.05*U[15] - 0.574852231579351*U[1]*U[3] - 1.73752196836555*U[1]*U[6] + 0.05*U[25] - 1.73752196836555*U[2]*U[7] - 0.0570836566449895*U[4] - 0.05*U[5]
	U_out[6] = -2.71889339738442*U[0]*U[7] - 2.71889339738442*U[10]*U[17] + 0.753865979381443*U[11]*U[15] + 0.753865979381443*U[12]*U[14] - 4.35022943581506*U[13]*U[19] + 0.05*U[16] + 0.753865979381443*U[1]*U[5] - 0.000777999678859866*U[20] + 0.753865979381443*U[2]*U[4] - 4.35022943581506*U[3]*U[9] - 0.05*U[6] + 0.0837128882242243*U[7]
	U_out[7] = 2.71889339738442*U[0]*U[6] + 2.71889339738442*U[10]*U[16] - 0.753865979381443*U[11]*U[14] + 0.753865979381443*U[12]*U[15] + 4.35022943581506*U[13]*U[18] + 0.05*U[17] - 0.753865979381443*U[1]*U[4] + 0.753865979381443*U[2]*U[5] + 4.35022943581506*U[3]*U[8] - 0.0837128882242243*U[6] - 0.05*U[7]
	U_out[8] = -2.26482546109569*U[0]*U[9] - 2.26482546109569*U[10]*U[19] - 1.7450294536311*U[13]*U[17] + 0.05*U[18] - 0.0017443933973331*U[21] - 1.7450294536311*U[3]*U[7] - 0.05*U[8] + 0.0603728636263923*U[9]
	U_out[9] = 2.26482546109569*U[0]*U[8] + 2.26482546109569*U[10]*U[18] + 1.7450294536311*U[13]*U[16] + 0.05*U[19] + 1.7450294536311*U[3]*U[6] - 0.0603728636263923*U[8] - 0.05*U[9]
	U_out[10] = 0.00454545454545455*U[0] - 0.0299589840451022*U[10] - 1.41868025576271*U[11]*U[2] + 1.41868025576271*U[12]*U[1] - 1.13494420461017*U[14]*U[5] + 1.13494420461017*U[15]*U[4] - 2.83736051152543*U[16]*U[7] + 2.83736051152543*U[17]*U[6] - 2.26988840922034*U[18]*U[9] + 2.26988840922034*U[19]*U[8] - 0.0153624025550847*U[21] - 0.0228186822802377*U[23] + 0.00807504401792934*U[29] + 0.00323001760717173*U[31] + 0.000468670500616653
	U_out[11] = -1.43757363347933*U[0]*U[12] + 1.02191932371371*U[10]*U[2] - 0.0352914755316095*U[11] + 0.0255954299692664*U[12] + 1.63507091794193*U[13]*U[5] + 1.21855791962175*U[14]*U[7] - 2.30011781356693*U[15]*U[3] - 1.21855791962175*U[15]*U[6] + 1.85472813238771*U[16]*U[5] - 1.85472813238771*U[17]*U[4] + 0.0105988967691095*U[1] + 0.00237875394324372*U[20] - 0.00388732494300134*U[28]
	U_out[12] = 1.43757363347933*U[0]*U[11] - 1.02191932371371*U[10]*U[1] - 0.0255954299692664*U[11] - 0.0352914755316095*U[12] - 1.63507091794193*U[13]*U[4] + 2.30011781356693*U[14]*U[3] - 1.21855791962175*U[14]*U[6] - 1.21855791962175*U[15]*U[7] + 1.85472813238771*U[16]*U[4] + 1.85472813238771*U[17]*U[5] - 0.0105988967691095*U[24] + 0.0105988967691095*U[2] + 0.00915929361228703*U[32]
	U_out[13] = -1.24843862507119*U[11]*U[5] + 1.24843862507119*U[12]*U[4] - 0.0385392017497231*U[13] - 2.3185288751322*U[14]*U[2] + 2.3185288751322*U[15]*U[1] - 2.49687725014237*U[16]*U[9] + 2.49687725014237*U[17]*U[8] - 4.63705775026441*U[18]*U[7] + 4.63705775026441*U[19]*U[6] + 0.00194123551627786*U[20] - 0.0231453954121832*U[22] - 0.00317233872132938*U[28] + 0.00571020969839289*U[30] + 0.0142857142857143*U[3]
	U_out[14] = -1.16886956037576*U[0]*U[15] + 0.422511733532696*U[10]*U[5] - 0.612715105162524*U[11]*U[7] - 1.38291034440645*U[12]*U[3] + 0.612715105162524*U[12]*U[6] + 1.79985224341047*U[13]*U[2] - 0.0419279046842654*U[14] + 0.0207014663040147*U[15] - 1.87294455066922*U[16]*U[2] + 1.87294455066922*U[17]*U[1] + 0.00598141266899433*U[21] - 0.00314405057531466*U[29] + 0.0181325685149777*U[4]
	U_out[15] = 1.16886956037576*U[0]*U[14] - 0.422511733532696*U[10]*U[4] + 1.38291034440645*U[11]*U[3] + 0.612715105162524*U[11]*U[6] + 0.612715105162524*U[12]*U[7] - 1.79985224341047*U[13]*U[1] - 0.0207014663040147*U[14] - 0.0419279046842654*U[15] - 1.87294455066922*U[16]*U[1] - 1.87294455066922*U[17]*U[2] - 0.0181325685149777*U[25] + 0.00740799464244248*U[33] + 0.0181325685149777*U[5]
	U_out[16] = -2.94535914360826*U[0]*U[17] + 0.569389237785845*U[10]*U[7] - 0.768581081081081*U[11]*U[5] - 0.768581081081081*U[12]*U[4] + 0.911022780457352*U[13]*U[9] + 1.42736486486486*U[14]*U[2] + 1.42736486486486*U[15]*U[1] - 0.0451998211991061*U[16] + 0.0365772529628368*U[17] - 4.71257462977322*U[19]*U[3] + 0.000339936796618951*U[20] - 0.000555519746922151*U[28] + 0.0218468468468468*U[6]
	U_out[17] = 2.94535914360826*U[0]*U[16] - 0.569389237785845*U[10]*U[6] + 0.768581081081081*U[11]*U[4] - 0.768581081081081*U[12]*U[5] - 0.911022780457352*U[13]*U[8] - 1.42736486486486*U[14]*U[1] + 1.42736486486486*U[15]*U[2] - 0.0365772529628368*U[16] - 0.0451998211991061*U[17] + 4.71257462977322*U[18]*U[3] + 0.0218468468468468*U[7]
	U_out[18] = -2.37660377951895*U[0]*U[19] + 0.0288656329496225*U[10]*U[9] + 1.50101291338039*U[13]*U[7] - 3.30992591155675*U[17]*U[3] - 0.0487836620662873*U[18] + 0.031291522765895*U[19] + 0.000904126828290182*U[21] - 0.000475242326846696*U[29] + 0.0259152215799615*U[8]
	U_out[19] = 2.37660377951895*U[0]*U[18] - 0.0288656329496225*U[10]*U[8] - 1.50101291338039*U[13]*U[6] + 3.30992591155675*U[16]*U[3] - 0.031291522765895*U[18] - 0.0487836620662873*U[19] + 0.0259152215799615*U[9]
	U_out[20] = 6.35094981132412e-8*U[11] + 8.50239961075093e-8*U[13] + 3.66419111790894e-8*U[16] - 6.35094981132412e-8*U[1] - 1.58263150747531e-7*U[20] - 0.000969739516006511*U[21]*U[24] - 0.00116937805032627*U[21]*U[26] - 0.00278561057700379*U[22]*U[25] - 0.00308506837848344*U[22]*U[27] - 0.00534779391583196*U[23]*U[26] - 7.91389255433123e-5*U[24] - 8.50239961075093e-8*U[3] - 3.66419111790894e-8*U[6]
	U_out[21] = 4.2438907259419e-8*U[0] - 4.2438907259419e-8*U[10] + 1.34106967724862e-7*U[14] + 5.07202450868017e-8*U[18] - 0.000708243330589963*U[20]*U[24] + 0.00172613939940631*U[20]*U[26] - 4.91188341087484e-7*U[21] - 0.00626988589506499*U[22]*U[24] - 0.0119940379417069*U[23]*U[25] - 7.9002862144845e-5*U[25] - 1.34106967724862e-7*U[4] - 5.07202450868017e-8*U[8]
	U_out[22] = -1.52343526260616e-7*U[13] - 0.00317032731211854*U[20]*U[25] + 0.0060425658523595*U[20]*U[27] + 0.00160884360522711*U[21]*U[24] - 1.04352693785129e-6*U[22] - 0.0178532887343688*U[23]*U[24] - 7.87771265478973e-5*U[26] + 1.52343526260616e-7*U[3]
	U_out[23] = 1.68596160851496e-8*U[0] - 1.68596160851496e-8*U[10] - 0.00857174744319731*U[20]*U[26] - 0.00140681174654471*U[21]*U[25] + 0.00871788567853858*U[22]*U[24] - 1.8115193420227e-6*U[23] - 7.84632552652771e-5*U[27]
	U_out[24] = -1.49531921957748e-7*U[12] + 0.00167798163428125*U[20]*U[21] + 7.9081381622411e-5*U[20] + 0.0046610600952257*U[21]*U[22] + 0.00913567778664237*U[22]*U[23] - 2.99063843915495e-7*U[24] + 1.49531921957748e-7*U[2]
	U_out[25] = -3.15752734584711e-7*U[15] + 0.0059559067658186*U[20]*U[22] + 0.0134007902230919*U[21]*U[23] + 7.89455158524528e-5*U[21] - 6.31505469169422e-7*U[25] + 3.15752734584711e-7*U[5]
	U_out[26] = -0.0005567719890759*U[20]*U[21] + 0.0139192997268975*U[20]*U[23] + 7.8720107381223e-5*U[22] - 1.18304364206348e-6*U[26]
	U_out[27] = -0.00295762797162433*U[20]*U[22] + 7.84066893926385e-5*U[23] - 1.94992690884148e-6*U[27]
	U_out[28] = -0.000962032794821447*U[11] - 0.00086613382185597*U[13] - 0.000192406558964289*U[16] + 0.975*U[21]*U[32] - 0.325*U[21]*U[34] + 1.3*U[22]*U[33] - 0.65*U[22]*U[35] + 1.625*U[23]*U[34] - 0.975*U[24]*U[29] - 1.3*U[25]*U[30] + 0.325*U[26]*U[29] - 1.625*U[26]*U[31] + 0.65*U[27]*U[30] - 0.00122336344718992*U[28]
	U_out[29] = 0.00173226764371194*U[10] - 0.000962032794821447*U[14] - 0.000192406558964289*U[18] - 0.975*U[20]*U[32] + 0.325*U[20]*U[34] + 1.625*U[22]*U[32] + 1.95*U[23]*U[33] + 0.975*U[24]*U[28] - 1.625*U[24]*U[30] - 1.95*U[25]*U[31] - 0.325*U[26]*U[28] - 0.00122336344718992*U[29] + 0.000137893034416115
	U_out[30] = 0.00155904087934075*U[13] - 1.3*U[20]*U[33] + 0.65*U[20]*U[35] - 1.625*U[21]*U[32] + 2.275*U[23]*U[32] + 1.625*U[24]*U[29] - 2.275*U[24]*U[31] + 1.3*U[25]*U[28] - 0.65*U[27]*U[28] - 0.00122336344718992*U[30]
	U_out[31] = 0.000692907057484776*U[10] - 1.625*U[20]*U[34] - 1.95*U[21]*U[33] - 2.275*U[22]*U[32] + 2.275*U[24]*U[30] + 1.95*U[25]*U[29] + 1.625*U[26]*U[28] - 0.00122336344718992*U[31] + 5.51572137664459e-5
	U_out[32] = 0.00226673637054264*U[12] + 0.975*U[20]*U[29] - 0.975*U[21]*U[28] + 1.625*U[21]*U[30] - 1.625*U[22]*U[29] + 2.275*U[22]*U[31] - 2.275*U[23]*U[30] - 0.00122336344718992*U[32]
	U_out[33] = 0.00226673637054264*U[15] + 1.3*U[20]*U[30] + 1.95*U[21]*U[31] - 1.3*U[22]*U[28] - 1.95*U[23]*U[29] - 0.00122336344718992*U[33]
	U_out[34] = -0.325*U[20]*U[29] + 1.625*U[20]*U[31] + 0.325*U[21]*U[28] - 1.625*U[23]*U[28] - 0.00122336344718992*U[34]
	U_out[35] = -0.65*U[20]*U[30] + 0.65*U[22]*U[28] - 0.00122336344718992*U[35]
	return U_out

In [ ]:
from qgs.integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
import matplotlib.pyplot as plt

In [ ]:
f_num, Df, ten_num = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

In [ ]:
integrator_num = RungeKuttaIntegrator()
integrator_num.set_func(f_num)

In [ ]:
ic = np.array([ 0.03099396, -0.00496147,  0.00173704, -0.00135697, -0.00772211,
       -0.01966693,  0.0133646 ,  0.001414  ,  0.01117828, -0.00041981,
        0.0296539 , -0.00059846, -0.0054299 ,  0.00265086, -0.00813694,
       -0.01703874,  0.00775618, -0.00437281,  0.00628166, -0.00235651,
       -0.00471975,  0.00143941, -0.00162457, -0.00087876,  0.00399296,
       -0.00265117, -0.00265747,  0.00141623, -0.0210313 ,  0.05039241,
        0.01445278,  0.00645707, -0.04806996, -0.06529819, -0.00455508,
        0.04942686])

In [ ]:
%%time
integrator.integrate(0., 1000000., 0.1, ic=ic, write_steps=5)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
%%time
integrator_num.integrate(0., 1000000., 0.1, ic=ic, write_steps=5)
reference_time_num, reference_traj_num = integrator_num.get_trajectories()

In [ ]:
varx = 21
vary = 29
plt.figure(figsize=(12, 10))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')
plt.plot(reference_traj_num[varx], reference_traj_num[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(reference_traj_num[varx, :] - reference_traj[varx, :])
plt.plot(reference_traj_num[vary, :] - reference_traj[vary, :])
plt.show()